In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import copy
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import pickle
import sys

import os
import torch
import torch.nn as nn
import torchvision.models as models
import time
import pandas as pd
import seaborn as sns
from PIL import Image
import os
import matplotlib.pyplot as plt
import cv2

from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable

from PIL import Image
import cv2
import albumentations as A

import time
import os
from tqdm.notebook import tqdm

from torchsummary import summary
import segmentation_models_pytorch as smp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np
from Pyfhel import Pyfhel

/home/cs-lab-12/.anaconda3/envs/he39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
clients = [0 , 1 ,2]

In [3]:
import numpy as np
from Pyfhel import Pyfhel

HE = Pyfhel()
ckks_params = {
    "scheme": "CKKS",
    "n": 2**14,  # Polynomial modulus degree. For CKKS, n/2 values can be
    "scale": 2**30,  # All the encodings will use it for float->fixed point
    "qi_sizes": [60, 30, 30, 30, 60],  # Number of bits of each prime in the chain.
}
HE.contextGen(**ckks_params)  # Generate context for ckks scheme
HE.keyGen()  # Key Generation: generates a pair of public/secret keys
HE.rotateKeyGen()

In [4]:
def generate_diffie_hellman_parameters():
    parameters = dh.generate_parameters(generator=2, key_size=512)
    return parameters


def generate_diffie_hellman_keys(parameters):
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key


def derive_key(private_key, peer_public_key):
    shared_key = private_key.exchange(peer_public_key)
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"handshake data",
    ).derive(shared_key)
    return derived_key


def encrypt_message_AES(key, message):
    serialized_obj = pickle.dumps(message)
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    encryptor = cipher.encryptor()
    padded_obj = serialized_obj + b" " * (16 - len(serialized_obj) % 16)
    ciphertext = encryptor.update(padded_obj) + encryptor.finalize()
    return ciphertext


def decrypt_message_AES(key, ciphertext):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    decryptor = cipher.decryptor()
    padded_obj = decryptor.update(ciphertext) + decryptor.finalize()
    serialized_obj = padded_obj.rstrip(b" ")
    obj = pickle.loads(serialized_obj)
    return obj


def setup_AES():
    num_clients = len(clients)
    parameters = generate_diffie_hellman_parameters()
    server_private_key, server_public_key = generate_diffie_hellman_keys(parameters)
    client_keys = [generate_diffie_hellman_keys(parameters) for _ in range(num_clients)]
    shared_keys = [
        derive_key(server_private_key, client_public_key)
        for _, client_public_key in client_keys
    ]
    client_shared_keys = [
        derive_key(client_private_key, server_public_key)
        for client_private_key, _ in client_keys
    ]

    return client_keys, shared_keys, client_shared_keys


client_keys, shared_keys, client_shared_keys = setup_AES()

In [5]:
def load_weights(model, weights):
    with torch.no_grad():
        for param, weight in zip(model.parameters(), weights):
            param.copy_(torch.tensor(weight))
    return model

In [6]:
def get_weights(model):
    return [param.cpu().detach().numpy() for param in model.parameters()]

In [7]:
def aggregate_wt(encypted_cwts):
    cwts = []
    for i, ecwt in enumerate(encypted_cwts):
        cwts.append(decrypt_message_AES(shared_keys[0], ecwt))
    # cwts = encypted_cwts
    resmodel = []
    for j in range(len(cwts[0])):  # for layers
        layer = []
        for k in range(len(cwts[0][j])):  # for chunks
            tmp = cwts[0][j][k].copy()
            for i in range(1, len(cwts)):  # for clients
                tmp = tmp + cwts[i][j][k]
            tmp = tmp / len(cwts)
            layer.append(tmp)
        resmodel.append(layer)

    res = [resmodel.copy() for _ in range(len(clients))]
    return res

In [8]:
def encrypt_wt(wtarray, i):
    cwt = []
    for layer in wtarray:
        flat_array = layer.astype(np.float64).flatten()

        chunks = np.array_split(flat_array, (len(flat_array) + 2**13 - 1) // 2**13)
        clayer = []
        for chunk in chunks:
            ptxt = HE.encodeFrac(chunk)
            ctxt = HE.encryptPtxt(ptxt)
            clayer.append(ctxt)
        cwt.append(clayer.copy())
    ciphertext = encrypt_message_AES(client_shared_keys[i], cwt)
    return ciphertext

In [9]:
def decrypt_weights(res):
    decrypted_weights = []
    for client_weights, model in zip(res, models):
        decrypted_client_weights = []
        wtarray = get_weights(model)
        for layer_weights, layer in zip(client_weights, wtarray):
            decrypted_layer_weights = []
            flat_array = layer.astype(np.float64).flatten()
            chunks = np.array_split(flat_array, (len(flat_array) + 2**13 - 1) // 2**13)
            for chunk, encrypted_chunk in zip(chunks, layer_weights):
                decrypted_chunk = HE.decryptFrac(encrypted_chunk)
                original_chunk_size = len(chunk)
                decrypted_chunk = decrypted_chunk[:original_chunk_size]
                decrypted_layer_weights.append(decrypted_chunk)
            decrypted_layer_weights = np.concatenate(decrypted_layer_weights, axis=0)
            decrypted_layer_weights = decrypted_layer_weights.reshape(layer.shape)
            decrypted_client_weights.append(decrypted_layer_weights)
        decrypted_weights.append(decrypted_client_weights)
    return decrypted_weights

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import DataLoader, Dataset
import time

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# Example encryption function (dummy encryption)
def encrypt_model_parameters(parameters):
    encrypted_params = [param**2 for param in parameters]
    return encrypted_params


# Define a simple text dataset
class SimpleTextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
        }


# Initialize the GPT-2 Small model and tokenizer
print("Loading GPT-2 Small model...")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set padding token
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
print("Model loaded successfully!")

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-5)

# Prepare dummy text dataset
print("Preparing dummy text dataset...")
texts = ["This is a sample text for GPT-2 Small."] * 50
dataset = SimpleTextDataset(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
print("Dummy text dataset prepared successfully!")


# Function to train the model for one epoch
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, batch in enumerate(dataloader, 0):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:  # Print every 10 batches
            print(f"[{i + 1}] loss: {running_loss / 10:.3f}")
            running_loss = 0.0
    return running_loss


# Measure training time without encryption
print("Training model for one epoch without encryption...")
start_time = time.time()
train_one_epoch(model, dataloader, criterion, optimizer, device)
end_time = time.time()
time_without_encryption = end_time - start_time
print(f"Training time without encryption: {time_without_encryption:.2f} seconds")

# Encrypt model parameters
print("Encrypting model parameters...")
model_parameters = [param.data.clone().detach() for param in model.parameters()]
start_encryption_time = time.time()
encrypted_parameters = encrypt_model_parameters(model_parameters)
end_encryption_time = time.time()
encryption_time = end_encryption_time - start_encryption_time
print(f"Encryption time: {encryption_time:.2f} seconds")

# Update model with encrypted parameters (dummy update for demonstration)
print("Updating model with encrypted parameters...")
for param, encrypted_param in zip(model.parameters(), encrypted_parameters):
    param.data = encrypted_param.to(device)
print("Model updated with encrypted parameters!")

# Measure training time with encryption
print("Training model for one epoch with encrypted parameters...")
start_time = time.time()
train_one_epoch(model, dataloader, criterion, optimizer, device)
end_time = time.time()
time_with_encryption = end_time - start_time
print(f"Training time with encryption: {time_with_encryption:.2f} seconds")

# Print final results
print("Final Results:")
print(f"Training time without encryption: {time_without_encryption:.2f} seconds")
print(f"Training time with encryption: {time_with_encryption:.2f} seconds")
print(f"Encryption time: {encryption_time:.2f} seconds")

Using device: cuda
Loading GPT-2 Small model...
Model loaded successfully!
Preparing dummy text dataset...
Dummy text dataset prepared successfully!
Training model for one epoch without encryption...
[10] loss: 3.940
Training time without encryption: 3.41 seconds
Encrypting model parameters...
Encryption time: 0.00 seconds
Updating model with encrypted parameters...
Model updated with encrypted parameters!
Training model for one epoch with encrypted parameters...
[10] loss: 10.617
Training time with encryption: 1.77 seconds
Final Results:
Training time without encryption: 3.41 seconds
Training time with encryption: 1.77 seconds
Encryption time: 0.00 seconds


In [11]:
start_encryption_time = time.time()
ew = encrypt_wt(get_weights(model), 0)
end_encryption_time = time.time()
encryption_time = end_encryption_time - start_encryption_time
print(f" encryption time : {encryption_time:.2f} seconds")

 encryption time : 703.87 seconds


In [12]:
ags_time = time.time()
res = aggregate_wt([ew for _ in range(10)])
age_time = time.time()  
print(f" aggregation time : {age_time - ags_time:.2f} seconds")

 aggregation time : 7264.64 seconds


In [13]:
start_dencryption_time = time.time()
ew = encrypt_wt(get_weights(model), 0)
end_encryption_time = time.time()
encryption_time = end_encryption_time - start_dencryption_time
print(f" dencryption time : {encryption_time:.2f} seconds")

 dencryption time : 994.67 seconds


In [14]:
import torch
import torchvision.models as models
import os


# Count the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters in      : {num_params}")

# Save the model to disk and get its size
model_path = "resnet50.pth"
torch.save(model.state_dict(), model_path)
model_size = os.path.getsize(model_path)
print(f"Size of the       model: {model_size / (1024 * 1024):.2f} MB")

# Cleanup
os.remove(model_path)

# Print results
print(f"Final Results:")
print(f"Number of parameters in : {num_params}")
print(f"Size of the : {model_size / (1024 * 1024):.2f} MB")

Number of parameters in      : 124439808
Size of the       model: 474.75 MB
Final Results:
Number of parameters in : 124439808
Size of the : 474.75 MB


In [15]:
import sys
size = sys.getsizeof(ew)
print(size)

16018104177


In [16]:
import pickle
import os


example_obj = model

# Save object to a pickle file
pickle_file = "example_obj.pkl"
with open(pickle_file, "wb") as file:
    pickle.dump(example_obj, file, protocol=pickle.HIGHEST_PROTOCOL)

# Check the size of the pickle file
file_size_bytes = os.path.getsize(pickle_file)
file_size_kb = file_size_bytes / 1024
file_size_mb = file_size_kb / 1024
file_size_gb = file_size_mb / 1024

print(f"Size of pickle file: {file_size_bytes} bytes")
print(f"Size of pickle file: {file_size_kb:.2f} KB")
print(f"Size of pickle file: {file_size_mb:.2f} MB")
print(f"Size of pickle file: {file_size_gb:.6f} GB")

Size of pickle file: 510421743 bytes
Size of pickle file: 498458.73 KB
Size of pickle file: 486.78 MB
Size of pickle file: 0.475367 GB


In [17]:
import pickle
import os


example_obj = ew

# Save object to a pickle file
pickle_file = "example_obj.pkl"
with open(pickle_file, "wb") as file:
    pickle.dump(example_obj, file, protocol=pickle.HIGHEST_PROTOCOL)

# Check the size of the pickle file
file_size_bytes = os.path.getsize(pickle_file)
file_size_kb = file_size_bytes / 1024
file_size_mb = file_size_kb / 1024
file_size_gb = file_size_mb / 1024

print(f"Size of pickle file: {file_size_bytes} bytes")
print(f"Size of pickle file: {file_size_kb:.2f} KB")
print(f"Size of pickle file: {file_size_mb:.2f} MB")
print(f"Size of pickle file: {file_size_gb:.6f} GB")

Size of pickle file: 16018104157 bytes
Size of pickle file: 15642679.84 KB
Size of pickle file: 15276.05 MB
Size of pickle file: 14.918022 GB
